In [59]:
import os 
import numpy as np 
import pandas as pd
import sys
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split

In [2]:
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import PIL
#%matplotlib inline

/home/melissa/miniconda3/envs/env2023/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/melissa/miniconda3/envs/env2023/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [70]:
wildtypes = ['S7068', 'S7070', 'S7071', 'S7074', 'S7086', 'S7087', 'S7091', 'S7098', 'S7101']
gaps = ['S7063', 'S7064', 'S7069', 'S7072', 'S7075', 'S7076', 'S7088', 'S7092', 'S7094', 'S7096']
all_ids = ['S7068', 'S7070', 'S7071', 'S7074', 'S7086', 'S7091', 'S7098', 'S7101',
          'S7063', 'S7064', 'S7069', 'S7072', 'S7075', 'S7076', 'S7088', 'S7092', 'S7094', 'S7096']
clean_br_states = '/home/melissa/PREPROCESSING/SYNGAP1/cleaned_br_files/'

In [71]:
os.chdir('/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Scripts/Preprocessing')
%run load_files.py
%run filter.py 
%run constants.py

directory_path = '/home/melissa/PREPROCESSING/SYNGAP1/numpyformat_baseline'
results_path = '/home/melissa/RESULTS/FINAL_MODEL/Rat/Power/'
error_path = '/home/melissa/RESULTS/XGBoost/SYNGAP1/ConnectivityErrors/'


syngap_1_ls = [ 'S7088', 'S7092', 'S7094', 'S7098', 'S7068', 'S7074', 'S7076', 'S7071', 'S7075', 'S7101']
syngap_2_ls =  ['S7091', 'S7070', 'S7072', 'S7083', 'S7063','S7064', 'S7069', 'S7086', 'S7091']

analysis_ls = ['S7101', 'S7088', 'S7092', 'S7094', 'S7098', 'S7068', 'S7074', 'S7076', 'S7071', 'S7075',
               'S7091', 'S7070', 'S7072', 'S7083', 'S7063','S7064', 'S7069', 'S7086', 'S7091', 'S7101']

In [72]:
#split into train and validation test to save images into separate folders
genotypes = ['wildtype' if id in wildtypes else 'gap' for id in all_ids]

# Split the IDs into training and validation sets, stratifying by genotype
train_ids, val_ids, train_genotypes, val_genotypes = train_test_split(
    all_ids, genotypes, test_size=0.2, stratify=genotypes, random_state=42)

In [77]:
train_directory = '/home/melissa/PROJECT_DIRECTORIES/SpectralSleepCNN/data/train/' 
val_directory = '/home/melissa/PROJECT_DIRECTORIES/SpectralSleepCNN/data/val/'

In [ ]:
for animal in all_ids:
    if animal in wildtypes:
        genotype = 'wt'
    elif animal in gaps:
        genotype = 'gap'
        
    clean_br_file = pd.read_pickle(clean_br_states +  f'{animal}_BL1.pkl')

    #extract indices only equal to 0, 1, 2
    clean_indices = clean_br_file[clean_br_file['brainstate'].isin([0,1,2])].index.to_list()

    
    load_files = LoadFiles(directory_path, animal)
    if animal in syngap_1_ls:
        data_1, brain_state_1 = load_files.load_one_analysis_file(start_times_dict = SYNGAP_baseline_start, end_times_dict = SYNGAP_baseline_end)
    elif animal in syngap_2_ls:
        data_1, data_2, brain_state_1, brain_state_2 = load_files.load_two_analysis_files(start_times_dict = SYNGAP_baseline_start, end_times_dict = SYNGAP_baseline_end)
    noise_filter_1 = NoiseFilter(data_1, brain_state_file = brain_state_1, channelvariables = channel_variables,ch_type = 'all')    
    bandpass_filtered_data_1 = noise_filter_1.filter_data_type()
    eeg_chan = bandpass_filtered_data_1[2, :]
    #emg_chan = bandpass_filtered_data_1[:, 14]
    
    split_eeg_data = np.split(eeg_chan, 17280, axis=0)
    #split_emg_data = np.split(emg_chan, 17280, axis = 1)
    
    clean_counter = 0  # Initialize counter for clean indices
    
    for idx, epoch in enumerate(split_eeg_data):
        if idx not in clean_indices:
            pass
        else:
            
            clean_counter += 1  # increment clean index counter
            
            file_row = clean_br_file.loc[idx]['brainstate']
            if file_row == 1:
                sleep_state = 'NREM'
            elif file_row == 0:
                sleep_state = 'wake'
            elif file_row == 2:
                sleep_state = 'REM'
            
            dpi = 100  # Dots per inch
            pixel_size = 224  # Desired size in pixels
            inch_size = pixel_size / dpi 
            
            f_eeg, t_eeg, Sxx_eeg = signal.spectrogram(epoch, fs=250.4, window='hann', scaling='spectrum')
             # Plotting spectrogram for EEG
            fig = plt.figure(figsize=[inch_size, inch_size])
            ax = fig.add_subplot(111)
            ax.axes.get_xaxis().set_visible(False)
            ax.axes.get_yaxis().set_visible(False)
            ax.set_frame_on(False)
            
            cax = ax.pcolormesh(t_eeg, f_eeg, 10 * np.log10(Sxx_eeg), shading='gouraud')
            ax.set_ylabel('Frequency [Hz]')
            ax.set_xlabel('Time [sec]')
            ax.set_title(f'EEG Spectrogram for Animal {animal} Epoch {idx} Brainstate {sleep_state}')
            fig.colorbar(cax, ax=ax, label='Intensity [dB]')
            ax.set_ylim(0, 20)  # Limit frequency range if desired
            if animal in train_ids:
                plt.savefig(train_directory + f'{animal}_{idx}_{genotype}_{sleep_state}.png')
                plt.close('all')
            elif animal in val_ids:
                plt.savefig(val_directory + f'{animal}_{idx}_{genotype}_{sleep_state}.png')
                plt.close('all')
            if clean_counter >= 1000:
                break  # Stop the loop
            